# Worksheet 1: Introduction to Z3

This is a worksheet for the first week of COMP3008.  There will be five worksheets in total, each designed to be completed within a week.  It is generally expected that a student will need more than the three computing class hours to complete a worksheet.

Working on these worksheets is an essential part of the module and a great opportunity to prepare for the coursework project and exam.  We strongly advice showing your solutions to any of the teaching staff to discuss your approach and receive feedback.


### Support

You are encouraged to discuss worksheet exercises with your classmates.  (However, you cannot discuss the individual project with anyone except teaching staff.)


## How to use Google Colab

Google Colab is a platform that allows one to run Python code in a web application.  This means that you do not need to install any software on your computer and that your code is stored in the cloud.

**You must make a copy of this notebook to your own Google Drive** (File -> Save a copy in Drive).  Changes that you make to this notebook will be lost once you close it!

## Z3 Theorem Prover

The first software package that we will use in this module is called Z3 Theorem Prover.  It was developed by Microsoft and is considered to be one of the most advanced solvers in its class.  Wikipedia has an [article](https://en.wikipedia.org/wiki/Z3_Theorem_Prover) with a brief overview of Z3.

The main purpose of Z3 is software verification, i.e. verification that a program will always work correctly.  (While it is impossible to answer this question in general -- you may have heard of the halting problem -- Z3 efficiently handles very complex cases.)  The language of Z3 is tailored for such tasks.  Nevertheless, Z3 is based on the first-order logic, and this is how we will use it.

You will have to learn the relevant aspects of the language of Z3 on your own; this worksheet only explains a few basics, and the documentation on the web is scarce.  The expectation is that you will study the rest by trial and error -- which is common for such specialised software packages.


## Installation of the Python package

Run the below command once, to install the Microsoft Z3 package.  This may take 20-30 seconds.

In [1]:
!pip install z3-solver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.1/29.1 MB 64.8 MB/s eta 0:00:00


Z3 Solver accepts knowledge bases in the SMT2 format.  In the next few exercises, you will use the SMT2 language to encode some simple knowledge.  You do not need to understand the Python code below; its only function is to pass the SMT2 program to the Z3 solver and then run Z3 to check if the program is satisfiable and print the model.

In [2]:
smt2program = """
(declare-const A Bool)
(declare-const B Bool)
(assert (and (xor A B) A))
"""

import z3

s = z3.Solver()
s.add(z3.parse_smt2_string(smt2program))
status = s.check()
print(status)
if status == z3.sat:
    print(s.model())

sat
[A = True, B = False]


## Exercises (part 1)

1. Spend a minute to carefully read the above program (the content of the `smt2program` variable).  It is in the so-called parenthesised prefix notation, known for being used in LISP -- one of the first functional programming languages.  In this notation, each pair of parentheses is a function call, with the first word in the parentheses being the function name and the rest of the items being parameters.  Pay attention to the third line (the `assert' command); can you identify the FOL formula encoded there?

2. Run the code snippet above.  The output tells you that the program is _satisfiable_, and also gives the values of variables $A$ and $B$ that satisfies it -- a model.  Check that these values satisfy the formula encoded in line 3 of the SMT2 program.

3. Read the following very brief reference for the SMT2 language.

    To comment out a line, place semicolon at the beginning of it:
    ```
    ; Z3 will ignore this line
    ```
    Every command in Z3 is a function call and has the following format:
    ```
    (<function name> <parameter 1> <parameter 2> ...)
    ```
    Each parameter can also be a function call.

    A few functions you may need:

    Function| Description
    --------|-----------
    `and`   | Logical and; can take more than two parameters
    `or`    | Logical or; can take more than two parameters
    `not`   | Logical not
    `=>`    | Implication ($\rightarrow$)
    `=`     | Equivalence ($\leftrightarrow$)
    `declare-const` | Declares a constant; first parameter is the constant name; second parameter is the constant type
    `assert` | Takes one parameter and demands that it evaluates to true


4. Translate each of the following formulas into the SMT2 language and use Z3 to find the values of the constants that satisfy each of them (edit the `smt2program` content).  Check the answers by substituting them into the original formulas.

   1. $A \lor \lnot B$
   1. $(A \lor \lnot B \lor \lnot C) \land B \land C$
   1. $((A \rightarrow B) \leftrightarrow (A \land B)) \land \lnot B$
   1. $((A \land \lnot B) \lor (\lnot A \land B)) \land (A \leftrightarrow B) \land \lnot (C \leftrightarrow B)$

   (Note that not all these formulas are satisfiable.)

In [25]:
smt2program = """
(declare-const A Bool)
(declare-const B Bool)
(declare-const C Bool)
(assert (and (or (and A (not B)) (and (not A) B)) (= A B) (not (= C B)) ))

"""

import z3

s = z3.Solver()
s.add(z3.parse_smt2_string(smt2program))
status = s.check()
print(status)
if status == z3.sat:
    print(s.model())

unsat


5. Z3 can handle non-Boolean variables.  For example, it can handle integers; just use `Int` instead of `Bool` when defining a constant.  The supported functions are `+`, `-`, `*`, `<`, `>`, etc.

   Check the following example of a program.  It defines two integer constants and requests that $x + y < 10$ and $x > 8$.  Note that we can write multiple `assert` commands to demand multiple formulas to be satisfied.


In [26]:
smt2program = """
(declare-const x Int)
(declare-const y Int)
(assert (< (+ x y) 10) )
(assert (> x 8 ))
"""

import z3

s = z3.Solver()
s.add(z3.parse_smt2_string(smt2program))
status = s.check()
print(status)
if status == z3.sat:
    print(s.model())

sat
[x = 9, y = 0]


6. Using Z3, find a solution to the following mathematical problem.

   > Find a number $x$ that is greater than 1 and that is divisible by 3 and 7.  


   There exist several solutions to this exercise; try finding one that does not require any new commands/functions.

In [35]:
smt2program = """
(declare-const x Int)
(assert (> x 1))
(assert (= (mod x 3) 0))
(assert (= (mod x 7) 0))
"""

import z3

s = z3.Solver()
s.add(z3.parse_smt2_string(smt2program))
status = s.check()
print(status)
if status == z3.sat:
    print(s.model())

sat
[x = 21]


7. Using Z3, find a solution to the following mathematical problem.  
   
   > There is a monotonically-increasing sequence of three integers from the interval $[1, 7]$; in other words, each next number is strictly greater than the previous number.
   We know that the first and the second numbers are even (divisible by 2).  We also know that the third number is the sum of the first and the second numbers.
   What are the numbers in the sequence?

In [37]:
smt2program = """
(declare-const x Int)
(declare-const y Int)
(declare-const z Int)
(assert (> x 1))
(assert (> y x))
(assert (> z y))
(assert (= (mod x 2) 0))
(assert (= (mod y 2) 0))
(assert (= (+ x y) z))
"""

import z3

s = z3.Solver()
s.add(z3.parse_smt2_string(smt2program))
status = s.check()
print(status)
if status == z3.sat:
    print(s.model())

sat
[x = 2, z = 6, y = 4]


## Encoding Z3 programs without SMT2

So far, you have seen that Z3 can magically solve mathematical problems, and you do not need to think about algorithms.  All you have to do is to formulate the requirements in a declarative language.  However you might have noticed that the SMT2 language (that language used by Z3) is not particularly user-friendly.

Now let us learn how to use Z3 directly from a Python program (without the SMT2 language).  The skill you will acquire is not just about this specific combination (Z3 and Python); it is about the concept of using a general-purpose solver such as Z3 from a general-purpose programming language such as Python.

Why Python?  It is concise and flexible which makes it ideal for working with external libraries.  It is a popular choice in applications of machine learning and other AI tools.

Some of you might not have experience in Python.  This is OK; knowing how to write programs in any procedural language such as Java and some minimal understanding of the Python syntax will be sufficient in this module.  Also, learning a bit of Python along the way will certainly be good for your studies and career.

Below is an example of a Python program that encodes a single rule $A \land \lnot B$ and calls Z3 to find the values of $A$ and $B$ that satisfy the rule.

In [ ]:
import z3

s = z3.Solver()

A = z3.Bool('A')
B = z3.Bool('B')
s.add(z3.And(A, z3.Not(B)))

if s.check() == z3.unsat:
    print('unsat')
else:
    print(f'A = {s.model().eval(A)}')
    print(f'B = {s.model().eval(B)}')

Read it carefully.  Note how Boolean constants are defined.  The `add` function is equivalent of `assert` in the Z3 language; it adds an expression to the knowledge base.  Functions `Not`, `And`, etc. are provided by the Z3 library to encode logical expressions.  Function `s.check()` executes the reasoning algorithm of Z3.  The result of the function is `sat` if the knowledge base is satisfiable and `unsat` if it is not.
It can also be `unknown` but this is unlikely to happen with simple examples.  The `s.model().eval` function gives you access to the model found by Z3; it returns the value of the given constant.

## Exercises (part 2)

8. Use the above approach to check satisfiability of the following formula:
   $$
   ((A \land \lnot B) \lor (\lnot A \land B)) \land (A \leftrightarrow B) \land \lnot (C \leftrightarrow B)
   $$

In [43]:
# Your solution to exercise 8

import z3

s = z3.Solver()
A = z3.Bool('A')
B = z3.Bool('B')
C = z3.Bool('C')
D = z3.Bool('D')
s.add(z3.And (z3.Or (z3.And(A, z3.Not(B))), (z3.And(z3.Not(A), B)), A == B, z3.Not(C == B)))

if s.check() == z3.unsat:
    print('unsat')
else:
    print(f'A = {s.model().eval(A)}')
    print(f'B = {s.model().eval(B)}')
    print(f'C = {s.model().eval(C)}')
    print(f'D = {s.model().eval(D)}')



unsat


9. Encode your solution to exercise 7 directly in Python (without SMT2).  To create an integer constant, you will need function `z3.Int` instead of `z3.Bool`.  You can use operators such as `>=`, `+`, etc. to compose Z3 expressions.  All the Z3 library functions are [described here](https://z3prover.github.io/api/html/namespacez3py.html#func-members).

In [48]:
# Your solution to exercise 9
"""(declare-const x Int)
(declare-const y Int)
(declare-const z Int)
(assert (> x 1))
(assert (> y x))
(assert (> z y))
(assert (= (mod x 2) 0))
(assert (= (mod y 2) 0))
(assert (= (+ x y) z))

s.add(x  2 == 0)
s.add(y mod 2 == 0)
"""

s = z3.Solver()
x = z3.Int('x')
y = z3.Int('y')
z = z3.Int('z')
s.add(x > 1)
s.add(y > x)
s.add(z > y)
s.add(x % 2 == 0)
s.add(y % 2 == 0)
s.add(x + y == z)

if s.check() == z3.unsat:
    print('unsat')
else:
    print(f'x = {s.model().eval(x)}')
    print(f'y = {s.model().eval(y)}')
    print(f'z = {s.model().eval(z)}')


x = 2
y = 4
z = 6


10.  Study the following Python script.  At what point does the program calculate $x + y$ and compare it to 4?  What exactly happens in line `s.add(x + y >= 4)`?  **It is crucial that you correctly understand the answer.**

In [ ]:


import z3

s = z3.Solver()
x = z3.Int('x')
y = z3.Int('y')
s.add(x + y >= 4)
if s.check() == z3.unsat:
    print('unsat')
else:
    print(f'x = {s.model().eval(x)}')
    print(f'y = {s.model().eval(y)}')

Your answer to exercise 10:

 The line s.add(x + y >= 4) adds a logical constraint to the solver’s knowledge base. The solver only evaluates whether this expression can be satisfied with some assignment of integers to x and y during s.check(), if satisfiable, s.model() gives concrete assignment that satisfies the proposed expression.